In [30]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# (1)------------------------ Load the Data----------------------------------
data = pd.read_excel("concrete_data.xlsx")

# Drop missing values

data = data.dropna()
# Separateing  feature from target

features = data[['cement', 'water', 'superplasticizer', 'age']].values

target = data['concrete_compressive_strength'].values

#(2)------------------  Preprocessing (Normalization) to be in the same scale-------------------

scaler = MinMaxScaler()

features_scaled = scaler.fit_transform(features)

# (3)-------------------Spliting train and test data 75% train and 25% test------------------------

X_train, X_test, y_train, y_test = train_test_split(features_scaled, target, test_size=0.25, random_state=42, shuffle=True)

#class NeuralNetwork that include all function we need feedforward and backward_propagation
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        # -------------------------Initialize weights with small random values multiplied by 0.01
        self.weights_input_hidden = np.random.randn(input_size, hidden_size) * 0.01

        self.weights_hidden_output = np.random.randn(hidden_size, output_size) * 0.01


    def sigmoid(self, x):
        # ---------------------------Sigmoid activation function: 1 / (1 + e^(-x))
        return 1 / (1 + np.exp(-x))

    def forward_propagation(self, x):
        # ------------Forward propagation through the network

        # ---------------(1) Calculate the input to the hidden layer
        hidden_input = np.dot(x, self.weights_input_hidden)


        # ----------------(2) Apply the sigmoid activation function to the hidden layer output
        hidden_output = self.sigmoid(hidden_input)


        # ................(3) Calculate the input to the output layer but not applied sigmoid activation to output
        output_input = np.dot(hidden_output, self.weights_hidden_output)

        
        return output_input, hidden_output

    def backward_propagation(self, x, y, learning_rate):
        # ---------------(1)get result of forward path 
        output, hidden_output = self.forward_propagation(x)

        # -----------------(2) calculate error 
        error = y - output

        #------------------(3) Backpropagation for the output layer
        output_error = error
        hidden_output_transposed = hidden_output.T
        self.weights_hidden_output += learning_rate * np.dot(hidden_output_transposed, output_error)

        # -----------------(4) Backpropagation for the hidden layer
        hidden_error = output_error.dot(self.weights_hidden_output.T)
        hidden_delta = hidden_error * hidden_output * (1 - hidden_output)

        x_transposed = x.T
        #------------------we dont applied sigmoid to output 
        self.weights_input_hidden += learning_rate * np.dot(x_transposed, hidden_delta)
     #----------------------traing to model -----------------------------------------------
    def train(self, X, y, epochs, learning_rate):
        for epoch in range(epochs):
            total_error = 0

            for i in range(len(X)):
                x = X[i]
                target = y[i]
                x = np.array([x])  # ------------------------Converting  it to 2D array 

                target = np.array([target])  #  ----------------Converting  it to 2D array 

                output, _ = self.forward_propagation(x)
                self.backward_propagation(x, target, learning_rate)

                total_error += 0.5 * np.mean((target - output) ** 2)
            #------------print in each 100 epoch-------------------------------------------------
            if epoch % 100 == 0:
                mean_squared_error = total_error / len(X)
                print(f"--------------------Epoch {epoch}, Mean Squared Error---------------: {mean_squared_error}")

    def predict(self, X):
        output, _ = self.forward_propagation(X)
        return output

    def calculate_error(self, predictions, targets):

        return mean_squared_error(targets, predictions)

#-----------(1) make  instance of the neural network
input_size = 4  #----------- features
hidden_size = 10  # ---------nodes in the hidden layer
output_size = 1  # ----------one output layer

NeuralNetwork_object = NeuralNetwork(input_size, hidden_size, output_size)

# --------------------------train model----------------------------------------
NeuralNetwork_object.train(X_train, y_train, epochs=1000, learning_rate=0.01)

# -----------------------predict result-------------------------------------------
predictions = NeuralNetwork_object.predict(X_test)

# ----------------------accuracy--------------------------------------------------
accuracy = NeuralNetwork_object.calculate_error(predictions, y_test)
print(f"--------------------Mean Squared Error on Test Set--------------- {accuracy}")

# Calculate R-squared on the test set
r2_value = r2_score(y_test, predictions)
print(f"---------------------R-squared on Test Set:---------------------{r2_value}")

print("----------------------predictions--------------------------------")

print(predictions)


--------------------Epoch 0, Mean Squared Error---------------: 118.43967831878923
--------------------Epoch 100, Mean Squared Error---------------: 41.683027895843914
--------------------Epoch 200, Mean Squared Error---------------: 39.60394276267629
--------------------Epoch 300, Mean Squared Error---------------: 38.80400076871173
--------------------Epoch 400, Mean Squared Error---------------: 35.96789507081341
--------------------Epoch 500, Mean Squared Error---------------: 33.98818844693023
--------------------Epoch 600, Mean Squared Error---------------: 33.26375567267479
--------------------Epoch 700, Mean Squared Error---------------: 32.892776191822435
--------------------Epoch 800, Mean Squared Error---------------: 32.02694294927011
--------------------Epoch 900, Mean Squared Error---------------: 30.90317312845452
--------------------Mean Squared Error on Test Set--------------- 58.54264310139771
---------------------R-squared on Test Set:---------------------0.823406846